# CS542 - Class Challenge - fine-grained classification of plants:

Our class challenge will consists of two tasks addressing an image recognition task where our dataset contains about 1K categories of plants with only about 250,000 images.  There will be two parts to this task:

1. Image classification. Imagine we have cateloged all the plants we care to identify, now we just need to create a classifier for them! Use your skills from the supervised learning sections of this course to try to address this problem.

2. Semi-Supervised/Few-Shot Learning.  Unfortunately, we missed some important plants we want to classify!  We do have some images we think contain the plant, but we have only have a few labels.  Our new goal is to develop an AI model that can learn from just these labeled examples.

Each student must submit a model on both tasks.  Students in the top 3 on each task will get 5% extra credit on this assignment.

This notebook is associated with the first task (image classification).


# Dataset
The dataset is downloaded on scc in the address: "/projectnb2/cs542-bap/classChallenge/data". You can find the python version of this notebook there as well or you could just type "jupyter nbconvert --to script baselineModel_task1.ipynb" and it will output "baselineModel_task1.py". You should be able to run "baselineModel_task1.py" on scc by simply typing "python baselineModel_task1.py"

Please don't try to change or delete the dataset.

# Evaluation:
You will compete with each other over your performance on the dedicated test set. The performance measure is top the 5 error, i.e: if the true class is in one of your top 5 likely predictions, then its error is 0, otherwise its error is 1.  So, your goal is to get an error of 0. This notebook outputs top5 accuracy, so it is 1 - top5 error.

# Baseline:
The following code is a baseline which you can use and improve to come up with your model for this task

# Suggestion
One simple suggestion would be to use a pretrained model on imagenet and finetune it on this data similar to this [link](https://keras.io/api/applications/)
Also you should likely train more than 2 epochs.

## Import TensorFlow and other libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(os.getenv("CUDA_VISIBLE_DEVICES"))
tf.config.set_soft_device_placement(True)
def get_n_cores():
  nslots = os.getenv('NSLOTS')
  if nslots is not None:
    return int(nslots)
  raise ValueError('Environment variable NSLOTS is not defined.')
print("NUM CORES: ", get_n_cores())
tf.config.threading.set_intra_op_parallelism_threads(get_n_cores()-1)
tf.config.threading.set_inter_op_parallelism_threads(1)

Num GPUs Available:  1
1
NUM CORES:  16


# Create a dataset

In [2]:
data_dir = '/projectnb2/cs542-bap/class_challenge/'

train_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'train_held_out_labeled.txt'))
train_unlabeled_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'train_held_out.txt'))
val_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'val_held_out.txt'))
test_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'test_held_out.txt'))

with open(os.path.join(data_dir, 'classes_held_out.txt'), 'r') as f:
  class_names = [c.strip() for c in f.readlines()]

num_classes = len(class_names)

## Write a short function that converts a file path to an (img, label) pair:

In [3]:
def decode_img(img, crop_size=224):
    img = tf.io.read_file(img)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [crop_size, crop_size])

def get_label(label):
    # find teh matching label
    one_hot = tf.where(tf.equal(label, class_names))
    # Integer encode the label
    return tf.reduce_min(one_hot)

def process_path(file_path):
    # should have two parts
    file_path = tf.strings.split(file_path)
    # second part has the class index
    label = get_label(file_path[1])
    # load the raw data from the file
    img = decode_img(tf.strings.join([data_dir, 'images/', file_path[0], '.jpg']))
    return img, label

def process_path_test(file_path):
    # load the raw data from the file
    img = decode_img(tf.strings.join([data_dir, 'images/', file_path, '.jpg']))
    return img, file_path

## Finish setting up data

In [4]:
batch_size = 32

In [5]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_unlabeled_ds = train_unlabeled_ds.map(process_path_test, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path_test, num_parallel_calls=AUTOTUNE)

## Data loader hyper-parameters for performance!

In [6]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
train_unlabeled_ds = configure_for_performance(train_unlabeled_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)


## A simple CNN model!

In [7]:
#DenseNet201 with initial imagenet weights, no top layers, global max pooling

DenseNet = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling='max',
)

#data augmentation layers
data_aug = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    layers.experimental.preprocessing.RandomTranslation(.2, .2, fill_mode='wrap'),
    layers.experimental.preprocessing.RandomContrast(.1),
    layers.experimental.preprocessing.RandomRotation(.2),
    layers.experimental.preprocessing.RandomZoom(.3)])

#model
model = tf.keras.Sequential([
    data_aug, 
    DenseNet,
    layers.Dropout(.5), 
    layers.Dense(num_classes)
])
inputs = layers.Input([None, None, 3], dtype=tf.uint8)
outputs = tf.cast(inputs, tf.float32)

#DenseNet data preprocessing
outputs = tf.keras.applications.densenet.preprocess_input(outputs)

outputs = model(outputs)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])


## The usual loss function

In [8]:
opt = keras.optimizers.Adam(learning_rate=.0001, amsgrad=True)
model.compile(
  optimizer=opt,
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy',tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)])

## Training

In [9]:
model.fit(train_ds,validation_data=val_ds,epochs=100,shuffle=True)

Epoch 1/100
4/4 [==============================] - 6s 1s/step - loss: 10.9413 - accuracy: 0.0500 - sparse_top_k_categorical_accuracy: 0.2100 - val_loss: 6.8611 - val_accuracy: 0.0611 - val_sparse_top_k_categorical_accuracy: 0.2426
Epoch 2/100
4/4 [==============================] - 2s 399ms/step - loss: 9.4985 - accuracy: 0.1200 - sparse_top_k_categorical_accuracy: 0.2700 - val_loss: 6.0676 - val_accuracy: 0.0776 - val_sparse_top_k_categorical_accuracy: 0.2739
Epoch 3/100
4/4 [==============================] - 2s 399ms/step - loss: 7.3496 - accuracy: 0.1100 - sparse_top_k_categorical_accuracy: 0.4300 - val_loss: 5.4650 - val_accuracy: 0.0974 - val_sparse_top_k_categorical_accuracy: 0.3218
Epoch 4/100
4/4 [==============================] - 2s 400ms/step - loss: 6.5949 - accuracy: 0.1100 - sparse_top_k_categorical_accuracy: 0.4600 - val_loss: 4.9283 - val_accuracy: 0.1106 - val_sparse_top_k_categorical_accuracy: 0.3944
Epoch 5/100
4/4 [==============================] - 2s 398ms/step - los

4/4 [==============================] - 2s 397ms/step - loss: 0.3335 - accuracy: 0.9200 - sparse_top_k_categorical_accuracy: 1.0000 - val_loss: 2.1881 - val_accuracy: 0.6007 - val_sparse_top_k_categorical_accuracy: 0.8581
Epoch 37/100
4/4 [==============================] - 2s 395ms/step - loss: 0.2525 - accuracy: 0.9000 - sparse_top_k_categorical_accuracy: 0.9900 - val_loss: 2.1961 - val_accuracy: 0.5990 - val_sparse_top_k_categorical_accuracy: 0.8597
Epoch 38/100
4/4 [==============================] - 2s 396ms/step - loss: 0.3545 - accuracy: 0.9000 - sparse_top_k_categorical_accuracy: 0.9900 - val_loss: 2.2256 - val_accuracy: 0.5908 - val_sparse_top_k_categorical_accuracy: 0.8630
Epoch 39/100
4/4 [==============================] - 2s 397ms/step - loss: 0.5075 - accuracy: 0.8800 - sparse_top_k_categorical_accuracy: 0.9800 - val_loss: 2.2358 - val_accuracy: 0.5842 - val_sparse_top_k_categorical_accuracy: 0.8597
Epoch 40/100
4/4 [==============================] - 2s 399ms/step - loss: 0.2

4/4 [==============================] - 2s 386ms/step - loss: 0.1895 - accuracy: 0.9500 - sparse_top_k_categorical_accuracy: 0.9900 - val_loss: 2.6139 - val_accuracy: 0.5990 - val_sparse_top_k_categorical_accuracy: 0.8680
Epoch 72/100
4/4 [==============================] - 2s 385ms/step - loss: 0.1498 - accuracy: 0.9500 - sparse_top_k_categorical_accuracy: 0.9900 - val_loss: 2.6528 - val_accuracy: 0.6073 - val_sparse_top_k_categorical_accuracy: 0.8630
Epoch 73/100
4/4 [==============================] - 2s 384ms/step - loss: 0.0660 - accuracy: 0.9800 - sparse_top_k_categorical_accuracy: 1.0000 - val_loss: 2.6334 - val_accuracy: 0.6139 - val_sparse_top_k_categorical_accuracy: 0.8696
Epoch 74/100
4/4 [==============================] - 2s 383ms/step - loss: 0.1073 - accuracy: 0.9400 - sparse_top_k_categorical_accuracy: 1.0000 - val_loss: 2.6083 - val_accuracy: 0.6073 - val_sparse_top_k_categorical_accuracy: 0.8663
Epoch 75/100
4/4 [==============================] - 2s 382ms/step - loss: 0.2

In [11]:
#add predictions of unlabeled data to labeled data and retrain

#copy data to a temp file to append predicted labels to
with open(os.path.join(data_dir, 'train_held_out_labeled.txt')) as f:
    lines = f.readlines()
    lines = [l for l in lines if "ROW" in l]
    with open("train_with_labeled_data.txt", "w") as f1:
        f1.writelines(lines)

#append predicted labels with high confidence to train dataset
trained_model = model
with open('train_with_labeled_data.txt', 'a') as f:
    for image_batch, image_names in train_unlabeled_ds:
        for image_name, predictions in zip(image_names.numpy(), trained_model.predict(image_batch)):
            inds = np.argmax(predictions)
            if predictions[inds] > .9:
                line = str(int(image_name)) + ' ' + class_names[inds]
                f.write(line + '\n')



In [12]:
#retrain model
train_ds = tf.data.TextLineDataset('train_with_labeled_data.txt') 
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_ds = configure_for_performance(train_ds)

model.fit(train_ds,validation_data=val_ds,epochs=50,shuffle=True)

Epoch 1/50
132/132 [==============================] - 32s 240ms/step - loss: 2.0412 - accuracy: 0.6153 - sparse_top_k_categorical_accuracy: 0.8916 - val_loss: 3.9467 - val_accuracy: 0.6205 - val_sparse_top_k_categorical_accuracy: 0.8977
Epoch 2/50
132/132 [==============================] - 28s 209ms/step - loss: 1.5484 - accuracy: 0.6492 - sparse_top_k_categorical_accuracy: 0.9166 - val_loss: 1.7480 - val_accuracy: 0.6568 - val_sparse_top_k_categorical_accuracy: 0.9191
Epoch 3/50
132/132 [==============================] - 28s 209ms/step - loss: 1.2370 - accuracy: 0.6904 - sparse_top_k_categorical_accuracy: 0.9337 - val_loss: 1.6436 - val_accuracy: 0.6419 - val_sparse_top_k_categorical_accuracy: 0.9307
Epoch 4/50
132/132 [==============================] - 28s 210ms/step - loss: 1.0783 - accuracy: 0.7267 - sparse_top_k_categorical_accuracy: 0.9451 - val_loss: 1.7171 - val_accuracy: 0.6452 - val_sparse_top_k_categorical_accuracy: 0.9274
Epoch 5/50
132/132 [==============================] 

132/132 [==============================] - 27s 208ms/step - loss: 0.1708 - accuracy: 0.9442 - sparse_top_k_categorical_accuracy: 0.9983 - val_loss: 1.7511 - val_accuracy: 0.7096 - val_sparse_top_k_categorical_accuracy: 0.9356
Epoch 36/50
132/132 [==============================] - 27s 208ms/step - loss: 0.1447 - accuracy: 0.9546 - sparse_top_k_categorical_accuracy: 0.9983 - val_loss: 1.9141 - val_accuracy: 0.6865 - val_sparse_top_k_categorical_accuracy: 0.9290
Epoch 37/50
132/132 [==============================] - 27s 208ms/step - loss: 0.1370 - accuracy: 0.9565 - sparse_top_k_categorical_accuracy: 0.9995 - val_loss: 2.0153 - val_accuracy: 0.6766 - val_sparse_top_k_categorical_accuracy: 0.9307
Epoch 38/50
132/132 [==============================] - 27s 208ms/step - loss: 0.1390 - accuracy: 0.9553 - sparse_top_k_categorical_accuracy: 0.9988 - val_loss: 2.1120 - val_accuracy: 0.6650 - val_sparse_top_k_categorical_accuracy: 0.9175
Epoch 39/50
132/132 [==============================] - 27s 2

# Output submission csv for Kaggle


In [14]:
with open('submission_task2_semisupervised.csv', 'w') as f:
  f.write('id,predicted\n')
  for image_batch, image_names in test_ds:
    predictions = model.predict(image_batch)
    for image_name, predictions in zip(image_names.numpy(), model.predict(image_batch)):
      inds = np.argmax(predictions)
      line = str(int(image_name)) + ',' + class_names[inds]
      f.write(line + '\n')

**Note**

Absolute path is recommended here. For example, use "/projectnb2/cs542-bap/[your directory name]/submission_task1_supervised.csv" to replace "submission_task1_supervised.csv".

Besides, you can request good resources by specify the type of gpus, such as "qsub -l gpus=1 -l gpu_type=P100 [your file name].qsub". This is helpful to avoid potential issues of GPUs, such as out of memory, etc.
